In [2]:
%load_ext autoreload
%autoreload 2

In [6]:
from absbox import API,save

#localAPI = API("http://localhost:8081")
#localAPI = API("https://absbox.org/api/latest")
localAPI = API("http://localhost:8081")

In [11]:
from absbox.local.china import SPV

test01 = SPV(
    "Senior/Junior"
    ,{"封包日":"2021-03-31","起息日":"2021-06-15","首次兑付日":"2021-07-26"
      ,"法定到期日":"2060-12-01","收款频率":"月末","付款频率":["每月",26]}
    ,{'清单':[["按揭贷款"
        ,{"放款金额":120,"放款利率":["固定",0.045],"初始期限":30
          ,"频率":"每月","类型":"等额本金","放款日":"2021-02-01"}
          ,{"当前余额":120
          ,"当前利率":0.08
          ,"剩余期限":20
          ,"状态":"正常"}]]}
    ,(("账户01",{"余额":0}),)
    ,(("A1",{"当前余额":100
             ,"当前利率":0.07
             ,"初始余额":100
             ,"初始利率":0.07
             ,"起息日":"2020-01-03"
             ,"利率":{"固定":0.08}
             ,"债券类型":{"过手摊还":None}})
      ,("B",{"当前余额":20
             ,"当前利率":0.0
             ,"初始余额":100
             ,"初始利率":0.07
             ,"起息日":"2020-01-03"
             ,"利率":{"固定":0.00}
             ,"债券类型":{"权益":None}
             }))
    ,(("信托费用",{"类型":{"固定费用":10}}),)
    ,{"未违约":[
         ["支付费用",["账户01"],['信托费用']]
         ,["支付利息","账户01",["A1"]]
         ,["支付本金","账户01",["A1"]]
         ,["支付本金","账户01",["B"]]
         ,["支付收益","账户01","B"]
     ]}
    ,(["利息回款","账户01"]
      ,["本金回款","账户01"]
      ,["早偿回款","账户01"]
      ,["回收回款","账户01"])
    ,None
    ,None
    ,None
)

In [13]:
result = localAPI.run(test01,
         assumptions=None,
         pricing={"贴现日":"2021-08-22","贴现曲线":[["2020-01-01",0.03],["2024-01-01",0.03]]},
         read=True)
result['pricing']

,估值,票面估值,WAL,久期,凸性,应计利息
A1,92.88,92.88,0.58,0.46,0.96,0.44
B,13.02,13.02,0.87,1.25,2.78,0.00


In [14]:
result = localAPI.run(test01,
         assumptions=None,
         pricing={"贴现日":"2021-08-25","贴现曲线":[["2020-01-01",0.03],["2024-01-01",0.03]]},
         read=True)
result['pricing']

,估值,票面估值,WAL,久期,凸性,应计利息
A1,92.92,92.92,0.57,0.46,0.94,0.5
B,13.02,13.02,0.86,1.24,2.76,0.0


In [15]:
result = localAPI.run(test01,
         assumptions=None,
         pricing={"贴现日":"2021-08-01","贴现曲线":[["2020-01-01",0.03],["2024-01-01",0.03]]},
         read=True)
result['pricing']

,估值,票面估值,WAL,久期,凸性,应计利息
A1,92.73,92.73,0.64,0.47,1.08,0.06
B,12.99,12.99,0.91,1.31,2.98,0.00


In [4]:
result = localAPI.run(test01,
         assumptions=None,
         pricing={"债券":{"A1":("2021-10-15",80)},"利率曲线":[["2020-01-01",0.03],["2024-01-01",0.03]]},
         read=True)
result['pricing']

,静态利差
A1,-0.001303


In [5]:
2200 * 0.05

110.0

In [20]:
from absbox.local.generic import Generic

test02 = Generic(
    "Multiple Waterfalls"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":20
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),)
    ,(("A1",{"balance":500
             ,"rate":0.07
             ,"originBalance":500
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("A2",{"balance":500
             ,"rate":0.07
             ,"originBalance":500
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}})
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30}}),)
    ,{"amortizing":[
         ["payFee",["acc01"],['trusteeFee']]
         ,["payInt","acc01",["A1","A2"]]
         ,["payPrin","acc01",["A1","A2"]]
         ,["payPrin","acc01",["B"]]
         ,["payResidual","acc01","B"]]
      ,"cleanUp":[]
      ,"endOfCollection":[]       # execute when collect money from pool
      ,("amortizing","defaulted"):[]   #execute when deal is `defaulted`
      ,("amortizing","accelerated"): #execute when deal is `accelerated`
         [["payFee",["acc01"],['trusteeFee']]
          ,["payInt","acc01",["A1","A2"]]
          ,["payPrin","acc01",["A1"]]
          ,["payPrin","acc01",["A2"]]
          ,["payPrin","acc01",["B"]]
          ,["payResidual","acc01","B"]]
      }
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None
    ,None
    ,{"AfterCollect":[
         (["all"
           ,[("cumPoolDefaultedRate",),">",0.05]
           ,[">","2021-11-15"]]
           ,("newStatus","Accelerated"))]}
    )

In [21]:
from absbox import API

localAPI = API("http://localhost:8081")

r = localAPI.run(test02,
           assumptions=[{"CPR":0.01}
                       ,{"CDR":0.1}
                       ,{"Recovery":(0.7,18)}],
           pricing={"PVDate":"2021-08-22"
                    ,"PVCurve":[["2021-01-01",0.025]
                               ,["2024-08-01",0.025]]},
           read=True)

In [22]:
r['result']['status']

,Date,From,To
0,2021-11-30,Amortizing,Accelerated


In [13]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum
Date,,,,,,,,,
2021-06-15,1826.53,313.46,23.09,5.19,54.82,0.00,0.00,0.045,None
2021-07-31,1706.03,103.27,6.78,1.49,15.74,0.00,0.00,0.045,None
2021-08-31,1586.75,102.65,6.33,1.44,15.19,0.00,0.00,0.045,None
2021-09-30,1469.25,102.03,5.89,1.34,14.13,0.00,0.00,0.045,None
2021-10-31,1353.94,101.45,5.45,1.20,12.66,0.00,0.00,0.045,None
2021-11-30,1239.90,100.84,5.02,1.14,12.06,0.00,0.00,0.045,None
2021-12-31,1127.94,100.26,4.60,1.01,10.69,0.00,0.00,0.045,None
2022-01-31,1017.30,99.65,4.18,0.95,10.04,0.00,0.00,0.045,None
2022-02-28,908.33,99.05,3.77,0.86,9.06,0.00,0.00,0.045,None


In [10]:
r['accounts']['acc01'].loc["2021-10-20"]

,balance,change,memo
date,,,
2021-10-20,109.26,0.00,"<SeqPayFee:[""trusteeFee""]>"
2021-10-20,106.50,-2.76,"<PayInt:[""A1"",""A2""],Nothing>"
2021-10-20,0.00,-106.50,"<PayPrin:[""A1"",""A2""],Nothing>"
2021-10-20,0.00,0.00,"<PayPrin:[""B""],Nothing>"
2021-10-20,0.00,0.00,"<PayYield:B,Nothing>"


In [11]:
r['accounts']['acc01'].loc["2021-11-20"]

,balance,change,memo
date,,,
2021-11-20,108.10,0.00,"<SeqPayFee:[""trusteeFee""]>"
2021-11-20,105.88,-2.22,"<PayInt:[""A1"",""A2""],Nothing>"
2021-11-20,0.00,-105.88,"<PayPrin:[""A1""],Nothing>"
2021-11-20,0.00,0.00,"<PayPrin:[""A2""],Nothing>"
2021-11-20,0.00,0.00,"<PayPrin:[""B""],Nothing>"
2021-11-20,0.00,0.00,"<PayYield:B,Nothing>"


In [12]:
r['accounts']['acc01'].loc["2022-02-20"]

,balance,change,memo
date,,,
2022-02-20,104.78,0.00,"<SeqPayFee:[""trusteeFee""]>"
2022-02-20,104.44,-0.34,"<PayInt:[""A1"",""A2""],Nothing>"
2022-02-20,104.44,0.00,"<PayPrin:[""A1""],Nothing>"
2022-02-20,46.24,-58.20,"<PayPrin:[""A2""],Nothing>"
2022-02-20,0.00,-46.24,"<PayPrin:[""B""],Nothing>"
2022-02-20,0.00,0.00,"<PayYield:B,Nothing>"
